In [19]:
import re
from keras.preprocessing.text import Tokenizer 

filename = "aesop.txt"

with open(filename, encoding='utf-8-sig') as f:
    text = f.read()
    
seq_length = 20
start_story = '|'*seq_length

# CLEANUP
text = text.lower()
text = start_story + text
text = text.replace('\n\n\n\n\n', start_story)

text = text.replace('\n', ' ')
text = re.sub(' +', '. ', text).strip()
text = text.replace('..', '.')

text = re.sub('([!"#$%&()*+,-/:;<=>?@[\]^_`{|}~])', r' \1', text)
text = re.sub('\s{2,}', ' ', text)

# Tokenization

tokenizer = Tokenizer(char_level = False, filters = '')
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
token_list = tokenizer.texts_to_sequences([text])[0]

In [20]:
print(text[:200])

 | | | | | | | | | | | | | | | | | | | |the . fox . and . the . grapes . a . hungry . fox . saw . some . fine . bunches . of . grapes . hanging . from . a . vine . that . was . trained . along . a . h


In [24]:
import numpy as np
from keras.utils import np_utils

def generate_sequences(token_list, step):
    X = []
    y = []
    for i in range(0, len(token_list) - seq_length, step):
        X.append(token_list[i: i + seq_length])
        y.append(token_list[i + seq_length])
    
    y = np_utils.to_categorical(y, num_classes = total_words)
    
    num_seq = len(X)
    print('Number of sequences:', num_seq, "\n")
    
    return X, y, num_seq


In [25]:
step = 1
seq_length = 20
X, y, num_seq = generate_sequences(token_list, step)

X = np.array(X)
y = np.array(y)

Number of sequences: 92889 



array([ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2, 26])